# 5. MDP를 모를 때 밸류 평가하기
바닥부터 배우는 강화 학습 5장에 있는 코드를 참고 했습니다.

In [ ]:
import random

from tqdm import trange

# Env & Agent

- GridWorld 클래스

In [ ]:
class GridWorld():
    def __init__(self):
        self.x=0
        self.y=0
    
    def step(self, a):
        # 0번 액션: 왼쪽, 1번 액션: 위, 2번 액션: 오른쪽, 3번 액션: 아래쪽
        if a==0:
            self.move_left()
        elif a==1:
            self.move_up()
        elif a==2:
            self.move_right()
        elif a==3:
            self.move_down()

        reward = -1 # 보상은 항상 -1로 고정
        done = self.is_done()
        return (self.x, self.y), reward, done

    def move_right(self):
        self.y += 1  
        if self.y > 3:
            self.y = 3
      
    def move_left(self):
        self.y -= 1
        if self.y < 0:
            self.y = 0
      
    def move_up(self):
        self.x -= 1
        if self.x < 0:
            self.x = 0
  
    def move_down(self):
        self.x += 1
        if self.x > 3:
            self.x = 3

    def is_done(self):
        if self.x == 3 and self.y == 3:
            return True
        else :
            return False

    def get_state(self):
        return (self.x, self.y)
      
    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

- Agent 클래스

In [ ]:
class Agent():
    def __init__(self):
        pass        

    def select_action(self):
        coin = random.random()
        if coin < 0.25:
            action = 0
        elif coin < 0.5:
            action = 1
        elif coin < 0.75:
            action = 2
        else:
            action = 3
        return action

# 5.1 몬테카를로 학습

- 초기 환경

In [ ]:
env = GridWorld()
agent = Agent()
data = [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
gamma = 1.0
alpha = 0.0001

- 5만번 에피소드 진행

In [ ]:
for k in trange(50000):
    done = False
    history = []
    # 에피소드 1회 진행
    while not done:
        action = agent.select_action()
        (x, y), reward, done = env.step(action)
        history.append((x, y, reward))
    # env 초기화
    env.reset()

    # 에피소드 종료 후 테이블 업데이트
    cum_reward = 0
    for transition in history[::-1]:  # history의 뒤쪽부터 차례차례 리턴을 계산
        x, y, reward = transition
        data[x][y] = data[x][y] + alpha * (cum_reward - data[x][y])
        cum_reward = cum_reward + gamma * reward

100%|██████████| 50000/50000 [00:08<00:00, 6031.56it/s]


- 학습이 끝난 후 데이터 출력해보기

In [ ]:
for row in data:
    print(row)

[-57.89461894626727, -55.75547098996772, -52.53417214895092, -49.44400118988403]
[-55.888189530167224, -53.0270203679075, -48.51446389965822, -43.63691406005128]
[-52.65479723408761, -48.82138553868068, -40.76660944653839, -30.05641024904144]
[-50.58896850030744, -45.371808937005774, -30.447043956992054, 0.0]


# 5.2 Temporal Difference 학습

- 초기 환경

In [ ]:
env = GridWorld()
agent = Agent()
data = [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
gamma = 1.0
alpha = 0.0001

- 5만번 에피소드 진행

In [ ]:
for k in trange(50000):
    done = False
    # 에피소드 1회 진행
    while not done:
        x, y = env.get_state()
        action = agent.select_action()
        (x_prime, y_prime), reward, done = env.step(action)
        x_prime, y_prime = env.get_state()

        # 한 번의 step이 진행되자 마자 바로 테이블 업데이트
        data[x][y] = data[x][y] + alpha * (reward + gamma * data[x_prime][y_prime] - data[x][y])
    env.reset()

100%|██████████| 50000/50000 [00:07<00:00, 6762.21it/s]


- 학습이 끝난 후 데이터 출력해보기

In [ ]:
for row in data:
    print(row)

[-20.66585134165552, -19.48128330712255, -17.819754351905324, -16.602143892787762]
[-19.50389727194165, -18.160644196405723, -16.09559887556902, -14.29095720254234]
[-17.864747556210865, -16.13806639248789, -12.959716310215018, -9.352614665592512]
[-16.630963591250303, -14.378239786086109, -9.423059965430737, 0]
